In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/worker_notebooks/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/worker_notebooks/clinical.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/worker_notebooks/typing.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
:0: FutureWarning: IPython widgets are experimental and may change in the future.


Populating the interactive namespace from numpy and matplotlib


"\ndef plot_cutoffs(gene, w):\n    kernel = sp.gaussian_kde(w[gene])\n    max_value = w[gene].max()\n    a = kernel(np.arange(0, max_value, max_value/float(100)))\n    coverage_values = np.arange(0, max_value, max_value/float(100))\n    plt.scatter(coverage_values, a)\n    for l in coverage_values[np.r_[True, a[1:] < a[:-1]] & np.r_[a[:-1] < a[1:], True]]:\n        plt.axvline(x=l, ymin=0, ymax=1, linewidth=2, color='k')\n    plt.title(gene)\n    plt.show()\n    plt.clf()\n"

###Getting the mhc/kir combinations that will be common enough to provide statistics on

In [8]:
df = pd.read_csv('/cellar/users/ramarty/Data/kir/biological/all_the_good_stuff.csv', index_col=0)
df = df[~(df.tissue.isin(['LIHC', 'DLBC', 'ACC', 'BRCA', 'UCS', 'SKCM']))]

In [9]:
df.columns

Index([u'Tumor_Sample_Barcode', u'num_neoantigens5', u'num_neoantigens10',
       u'total_presented_peptides', u'mutation_count', u'max_neoantigen',
       u'num_neoantigens1', u'KIR2DP1', u'KIR2DS4', u'KIR2DS5', u'KIR2DS2',
       u'KIR2DS3', u'KIR2DS1', u'KIR2DL3', u'KIR2DL2', u'KIR2DL1', u'KIR2DL4',
       u'KIR3DS1', u'KIR3DL1', u'KIR3DL2', u'KIR3DL3', u'KIR2DL5A', u'KIR3DP1',
       u'Sample', u'barcode', u'age_at_diagnosis', u'days_to_death',
       u'days_to_last_followup', u'ethnicity', u'gender', u'race', u'tissue',
       u'vital_status', u'Bcells', u'Monocytes', u'Neutrophils', u'NKcells',
       u'Tcells', u'unkCells', u'A1', u'A2', u'B1', u'B2', u'C1', u'C2',
       u'A1_', u'A2_', u'B1_', u'B2_', u'C1_', u'C2_', u'activating',
       u'inhibitory', u'ratio', u'capture_kit'],
      dtype='object')

In [10]:
hla_c = ['C1', 'C2']
hla_b = ['B1', 'B2']
hla_a = ['A1', 'A2']
sample = ['Tumor_Sample_Barcode']

In [11]:
for hla, name in ([[hla_a, 'hla_a'], [hla_b, 'hla_b'], [hla_c, 'hla_c']]):
    x = df[hla + sample].groupby(hla).count().reset_index().sort('Tumor_Sample_Barcode', ascending=False)
    x[x.Tumor_Sample_Barcode > 20].to_csv('/cellar/users/ramarty/Data/kir/biological/combinations.{0}.csv'.format(name))

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from IPython.kernel.zmq import kernelapp as app


Also get the frequent singles.

In [25]:
As = list(pd.Series(list(df['A1']) + list(df['A2'])).value_counts()[:15].index)
Bs = list(pd.Series(list(df['B1']) + list(df['B2'])).value_counts()[:23].index)
Cs = list(pd.Series(list(df['C1']) + list(df['C2'])).value_counts()[:13].index)

In [26]:
for group, name in ([[As, 'A'], [Bs, 'B'], [Cs, 'C']]):
    with open('/cellar/users/ramarty/Data/kir/biological/singles.{0}.txt'.format(name), 'w') as out_file:
        for t in group:
            out_file.write('{0}\n'.format(t))

May also want to integrate allele calls with four digits - need to go back to the original document for that

In [24]:
As = list(pd.Series(list(df['A1_']) + list(df['A2_'])).value_counts()[:15].index)
Bs = list(pd.Series(list(df['B1_']) + list(df['B2_'])).value_counts()[:23].index)
Cs = list(pd.Series(list(df['C1_']) + list(df['C2_'])).value_counts()[:21].index)

In [26]:
for group, name in ([[As, 'A'], [Bs, 'B'], [Cs, 'C']]):
    with open('/cellar/users/ramarty/Data/kir/biological/singles.{0}.four_digit.txt'.format(name), 'w') as out_file:
        for t in group:
            out_file.write('{0}\n'.format(t))